# Data Preprocessing for PetFinder6000

In [5]:
!pip install --upgrade pip

#!pip install tensorflow
%conda install tensorflow

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - tensorflow


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _tflow_select-2.3.0        |            eigen           3 KB
    absl-py-1.3.0              |   py37h06a4308_0         166 KB
    aiohttp-3.8.1              |   py37h7f8727e_1         514 KB
    aiosignal-1.2.0            |     pyhd3eb1b0_0          12 KB
    astor-0.8.1                |   py37h06a4308_0          47 KB
    astunparse-1.6.3           |             py_0          17 KB
    async-timeout-4.0.2        |   py37h06a4308_0          11 KB
    asynctest-0.13.0           |             py_0          26 KB
    blinker-1.4 

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import os
import glob
import shutil

import boto3
import sagemaker

## Cats

## Load Data from S3

In [2]:
%env AWS_PROFILE=aeroxye-sagemaker

env: AWS_PROFILE=aeroxye-sagemaker


In [3]:
!aws sts get-caller-identity

{
    "UserId": "AROAWC4YSIQL5OBFCNGEX:botocore-session-1686895886",
    "Account": "418542404631",
    "Arn": "arn:aws:sts::418542404631:assumed-role/SageMaker-UserRole/botocore-session-1686895886"
}


In [4]:
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='SageMaker-UserRole')['Role']['Arn']

Couldn't call 'get_role' to get Role ARN from role name SageMaker-UserRole to get Role path.


In [5]:
region = boto3.Session().region_name

data_bucket = "dynamodbpetfinder"
object_prefix = "cat_"
local_path = "../data/cat/"

if not os.path.exists(local_path):
    os.makedirs(local_path)
else:
    shutil.rmtree(local_path)
    os.makedirs(local_path)

pattern = r'[0-9]+'
s3 = boto3.client("s3")

result = s3.list_objects(Bucket=data_bucket, Prefix=object_prefix, Delimiter="/")
subfolders = [re.search(pattern, o.get("Prefix")).group() for o in result.get("CommonPrefixes")]
subfolders.sort(reverse=True)

object_path = object_prefix + subfolders[0] + "/"
files = s3.list_objects(Bucket=data_bucket, Prefix=object_path, Delimiter="/")

pattern = rf'{object_path}(.+)'
for content in files.get("Contents"):
    file_path = content.get("Key")
    filename = re.findall(pattern, file_path)[0]
    print(filename)

    with open(local_path+filename, 'wb') as file:
        s3.download_fileobj(
            Bucket=data_bucket,
            Key=file_path,
            Fileobj=file
        )

run-1686726820579-part-r-00000
run-1686726820579-part-r-00001
run-1686726820579-part-r-00002
run-1686726820579-part-r-00003


In [6]:
file_list = glob.glob(local_path + "*")

print(file_list)

dfs = [] # an empty list to store the data frames
for file in file_list:
    data = pd.read_csv(file) # read data frame from csv file
    dfs.append(data) # append the data frame to the list
    
cats = pd.concat(dfs, ignore_index=True)

['../data/cat\\run-1686726820579-part-r-00000', '../data/cat\\run-1686726820579-part-r-00001', '../data/cat\\run-1686726820579-part-r-00002', '../data/cat\\run-1686726820579-part-r-00003']


In [51]:
# cats = pd.read_csv('../data/cats_5-23-2023.csv', header=0)

In [52]:
cats.head()

,id,cws_id,C_othercats,C_kids,C_primarycolor,name,C_employment,sweet,C_homeownership,active,...,quiet,C_otherdogs,C_attention,loving,updatedAt,C_adoptionfee,C_sweetspicy,description,spicy,C_energy
0,54b908dd-a325-459a-bdcb-5b1901d8324e,1681491254-120,-1,0,White,BoyBoy the oriental mix tabby cat,no preference,True,owned,False,...,True,0,independent,True,2023-05-15T02:51:11.735Z,yes,sweet,"quiet, loving, sweet, shy",False,chill
1,1a929522-33b0-42f2-8a80-a6f097a0e1fe,1681491547-251,1,0,Tabby,Mike and Bella,working full time,True,no preference,True,...,False,0,NaN,True,2023-05-15T02:51:14.961Z,no preference,sweet,"playful, active, loving, sweet",False,high energy
2,b3538eec-3e8b-4c48-9adf-08c5c5fbbfce,1681491592-271,0,0,White,Maya for adoption,student,False,parents,False,...,False,0,NaN,False,2023-05-15T02:51:19.367Z,yes,NaN,NaN,False,chill
3,b79b5e65-621b-4906-aea4-fffef2259fec,1681491440-203,1,0,Calico/Tortie,Sally – 8 mths old tortoiseshell,working full time,False,owned,False,...,True,0,independent,True,2023-05-15T02:51:36.655Z,no preference,sweet,"quiet, loving, friendly",False,chill
4,6117e9df-fa0f-43dd-b6cb-b62a8748bfd8,1681491915-414,0,0,Tabby,Matcha,working full time,True,owned,False,...,False,0,NaN,True,2023-05-15T02:51:50.223Z,yes,sweet,"loving, sweet",False,chill


## Process Data

In [7]:
# rename headers
cl_cats = cats.rename(columns={'C_gender': 'gender',
                              'C_primarycolor': 'primary_color',
                              'C_agegroup': 'age_group',
                              'C_energy': 'energy_level',
                              'C_attention': 'attention_need',
                              'C_sweetspicy': 'personality',
                              'C_firstcat': 'good_first_cat',
                              'C_othercats': 'good_with_other_cats',
                              'C_otherdogs': 'good_with_other_dogs',
                              'C_kids': 'good_with_kids',
                              'C_employment': 'preferred_employment',
                              'C_homeownership': 'preferred_home_ownership',
                              'C_allergies': 'good_with_allergies',
                              'C_adoptionfee': 'require_fee',
                              'createdAt': 'created_at',
                              'updatedAt': 'updated_at',
                              })

In [8]:
# fill attention and personality columns as neutral
cl_cats['attention_need'] = cl_cats['attention_need'].fillna('neutral')
cl_cats['personality'] = cl_cats['personality'].fillna('neutral')

# fill empty description with no description available
cl_cats['description'] = cl_cats['description'].fillna('no description available')

In [9]:
# map integers in categorical columns
def map_categorical(val):
    if val == 1:
        return 'yes'
    elif val == -1:
        return 'no'
    else:
        return 'neutral'

cl_cats['good_first_cat'] = cl_cats['good_first_cat'].map(lambda val: map_categorical(val))
cl_cats['good_with_other_cats'] = cl_cats['good_with_other_cats'].map(lambda val: map_categorical(val))
cl_cats['good_with_other_dogs'] = cl_cats['good_with_other_dogs'].map(lambda val: map_categorical(val))
cl_cats['good_with_kids'] = cl_cats['good_with_kids'].map(lambda val: map_categorical(val))

In [10]:
# convert string fields to lower case (gender, breed, primary_color, age_group, energy_level, attention_need, personality, preferred_employment, preferred_home_ownership, require_fee)
cl_cats['gender'] = cl_cats['gender'].map(lambda val: val.lower())
cl_cats['breed'] = cl_cats['breed'].map(lambda val: val.lower())
cl_cats['primary_color'] = cl_cats['primary_color'].map(lambda val: val.lower())
cl_cats['age_group'] = cl_cats['age_group'].map(lambda val: val.lower())
cl_cats['energy_level'] = cl_cats['energy_level'].map(lambda val: val.lower())
cl_cats['attention_need'] = cl_cats['attention_need'].map(lambda val: val.lower())
cl_cats['personality'] = cl_cats['personality'].map(lambda val: val.lower())
cl_cats['preferred_employment'] = cl_cats['preferred_employment'].map(lambda val: val.lower())
cl_cats['preferred_home_ownership'] = cl_cats['preferred_home_ownership'].map(lambda val: val.lower())
cl_cats['require_fee'] = cl_cats['require_fee'].map(lambda val: val.lower())

cl_cats['description'] = cl_cats['description'].map(lambda val: val.encode("ascii", "ignore").decode().replace('\n', ' ').replace('\r', '').lower())
cl_cats['details'] = cl_cats['details'].map(lambda val: val.encode("ascii", "ignore").decode().replace('\n', ' ').replace('\r', '').lower())

In [11]:
# convert types
# cl_cats = cl_cats.astype({'gender': 'category',
#                           'breed': 'category',
#                           'primary_color': 'category',
#                           'age_group': 'category',
#                           'energy_level': 'category',
#                           'attention_need': 'category',
#                           'personality': 'category',
#                           'good_first_cat': 'category',
#                           'good_with_other_cats': 'category',
#                           'good_with_other_dogs': 'category',
#                           'good_with_kids': 'category',
#                           'preferred_employment': 'category',
#                           'preferred_home_ownership': 'category',
#                           'good_with_allergies': 'category',
#                           'require_fee': 'category',
#                           })

cl_cats['created_at'] = pd.to_datetime(cl_cats['created_at']).dt.strftime('%Y-%m-%dT%H:%M:%SZ')
cl_cats['updated_at'] = pd.to_datetime(cl_cats['updated_at']).dt.strftime('%Y-%m-%dT%H:%M:%SZ')

cl_cats = cl_cats.astype({'playful': 'int',
                          'active': 'int',
                          'curious': 'int',
                          'talkative': 'int',
                          'quiet': 'int',
                          'loving': 'int',
                          'sweet': 'int',
                          'likes_held': 'int',
                          'friendly': 'int',
                          'shy': 'int',
                          'spicy': 'int',
                          'loves_attention': 'int'
                          })

In [12]:
# drop glue columns
cl_cats = cl_cats.drop(['__typename', '_lastChangedAt', '_version'], axis=1)

In [13]:
cl_cats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404 entries, 0 to 403
Data columns (total 37 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   id                        404 non-null    object
 1   cws_id                    404 non-null    object
 2   good_with_other_cats      404 non-null    object
 3   good_with_kids            404 non-null    object
 4   primary_color             404 non-null    object
 5   name                      404 non-null    object
 6   preferred_employment      404 non-null    object
 7   sweet                     404 non-null    int32 
 8   preferred_home_ownership  404 non-null    object
 9   active                    404 non-null    int32 
 10  loves_attention           404 non-null    int32 
 11  likes_held                404 non-null    int32 
 12  gender                    404 non-null    object
 13  good_first_cat            404 non-null    object
 14  talkative                 

In [14]:
cl_cats = cl_cats.set_index('id')

In [15]:
cl_cats.head()

,cws_id,good_with_other_cats,good_with_kids,primary_color,name,preferred_employment,sweet,preferred_home_ownership,active,loves_attention,...,quiet,good_with_other_dogs,attention_need,loving,updated_at,require_fee,personality,description,spicy,energy_level
id,,,,,,,,,,,,,,,,,,,,,
4f5637ed-1bb9-4e51-979f-94213f0fbd63,1681491605-277,yes,neutral,ginger,Ginger siblings looking for their purr-ever home!,student,1,parents,0,0,...,0,neutral,neutral,1,2023-05-15T02:51:17Z,yes,sweet,"playful, curious, loving, sweet",0,high energy
6b33d102-9497-43c7-a7a3-a28d6a9d1880,1681491000-6,no,neutral,calico/tortie,Nana the Calicococo,no preference,1,owned,0,0,...,0,neutral,neutral,0,2023-05-15T02:51:07Z,yes,sweet,sweet,0,chill
eec13c36-d5a2-4802-a508-517c2753663b,1681491462-213,neutral,neutral,tabby,Adorable and Affectionate Junior,working part time,1,rental with landlord’s consent,0,0,...,0,neutral,neutral,1,2023-05-15T02:51:37Z,yes,sweet,"curious, loving, sweet",0,chill
8843e8e2-27c5-4bdf-966f-5e2cfc18a351,1681491248-117,neutral,neutral,ginger,Luna- The Lap Cat,no preference,1,owned,0,0,...,0,neutral,neutral,1,2023-05-15T02:51:28Z,yes,sweet,"loving, sweet, friendly",0,chill
d1f210b0-fd38-43fa-9f6b-880a8e2bf4de,1681491181-87,no,neutral,ginger,Jimmy,not working,1,owned,1,0,...,0,no,neutral,1,2023-05-15T02:51:11Z,yes,sweet,"playful, active, loving, sweet",0,high energy


## Export Processed Data as csv to Local Folder

In [16]:
processed_folder = "../data/processed/auxiliary/"

if not os.path.exists(processed_folder):
    os.makedirs(processed_folder)

cl_cats.to_csv(f'{processed_folder}cats.csv')

## Cat Profile Images

In [ ]:
%pip install -U opencv-python-headless

In [17]:
import cv2

In [18]:
new_size = 128

In [19]:
data_bucket = "petfinder6000images"
local_path = "../data/cat_images/"

if not os.path.exists(local_path):
    os.makedirs(local_path)
else:
    shutil.rmtree(local_path)
    os.makedirs(local_path)
    
s3 = boto3.client("s3")

In [20]:
def vectorise(cws_id):
    filename = f'cropped_{cws_id}.jpg'
    s3.download_file(
        Bucket=data_bucket,
        Key=filename,
        Filename=local_path+filename
    )
    
    img = cv2.imread(local_path+filename)
    try:
        resize = cv2.resize(img, (new_size, new_size))
        resize = cv2.cvtColor(resize,cv2.COLOR_BGR2RGB)
        resize = np.array(resize.tolist())/255.0
        return resize
        # img_flat = resize.reshape(-1)
        # return str(np.array2string(img_flat, precision=2, separator=',', suppress_small=True)).encode('utf-8')
    except:
        print(file)

In [22]:
cat_images = cl_cats.reset_index()
cat_images = cat_images.loc[:, ['id', 'cws_id', 'updated_at']]
cat_images['img_shape'] = str([new_size, new_size, 3])
cat_images['img_vector'] = cat_images['cws_id'].map(lambda cws_id: vectorise(cws_id))

In [23]:
img_vectors = cat_images['img_vector'].values

In [24]:
# Using a pretrained model to generate features

#### Importing pretrained MobileNetV2
mnetv2_base = tf.keras.applications.mobilenet_v2.MobileNetV2(input_shape=(new_size,new_size,3), include_top=False, weights='imagenet')

# Freezing layers
for layer in mnetv2_base.layers:
    layer.trainable = False

    
# defining a function to extract features
def img_feature_extraction(img_vectors, pre_model):

    batch_img = []
    
    # preprocessing and then using pretrained model to extract features
    for image in img_vectors:
        im_toarray = tf.keras.preprocessing.image.img_to_array(image)
        
        im_toarray = np.expand_dims(image, axis=0)
        im_toarray = tf.keras.applications.mobilenet.preprocess_input(im_toarray)
        
        batch_img.append(im_toarray)
        
    batch_img = np.vstack(batch_img)
    features = pre_model.predict(batch_img, batch_size=64)
    features= features.reshape((len(img_vectors), -1))
    return features

In [25]:
# Extract features
features = img_feature_extraction(img_vectors, mnetv2_base)

features.shape

7/7 [==============================] - 2s 169ms/step


(404, 20480)

In [28]:
# add the feature vectors to dataframe
feature_vectors = pd.Series(features.tolist(), index = cat_images.index)
cat_images = cat_images.merge(feature_vectors.rename('feature_vectors'), left_index=True, right_index=True)

In [29]:
# stringify vectors
cat_images['feature_vectors']=cat_images['feature_vectors'].astype(pd.StringDtype())

cat_images['img_vector']=cat_images['img_vector'].apply(lambda x: x.reshape(-1).tolist())
cat_images['img_vector']=cat_images['img_vector'].astype(pd.StringDtype())

In [30]:
cat_images.head()

,id,cws_id,updated_at,img_shape,img_vector,feature_vectors
0,4f5637ed-1bb9-4e51-979f-94213f0fbd63,1681491605-277,2023-05-15T02:51:17Z,"[128, 128, 3]","[-0.9944329104190696, -0.9947097270280661, -0....","[0.0, 2.044503927230835, 0.0, 0.0, 0.0, 0.0, 0..."
1,6b33d102-9497-43c7-a7a3-a28d6a9d1880,1681491000-6,2023-05-15T02:51:07Z,"[128, 128, 3]","[-0.99520184544406, -0.9951710880430603, -0.99...","[0.0, 2.0194919109344482, 0.0, 0.0, 0.0, 0.0, ..."
2,eec13c36-d5a2-4802-a508-517c2753663b,1681491462-213,2023-05-15T02:51:37Z,"[128, 128, 3]","[-0.9948327566320646, -0.9943713956170703, -0....","[0.0, 2.085791826248169, 0.0, 0.0, 0.0, 0.0, 0..."
3,8843e8e2-27c5-4bdf-966f-5e2cfc18a351,1681491248-117,2023-05-15T02:51:28Z,"[128, 128, 3]","[-0.9923106497500961, -0.9921876201460976, -0....","[0.0, 2.0698859691619873, 0.0, 0.0, 0.0, 0.0, ..."
4,d1f210b0-fd38-43fa-9f6b-880a8e2bf4de,1681491181-87,2023-05-15T02:51:11Z,"[128, 128, 3]","[-0.9969550173010381, -0.9965244136870435, -0....","[0.0, 2.1150710582733154, 0.0, 0.0, 0.0, 0.0, ..."


In [31]:
cat_images.dtypes

id                 object
cws_id             object
updated_at         object
img_shape          object
img_vector         string
feature_vectors    string
dtype: object

In [32]:
def cast_object_to_string(data_frame):
    """
    Cast all columns of data_frame of type object to type string and return it.
    Parameters:
        data_frame: A pandas Dataframe
    Returns:
        Data frame
    """
    for label in data_frame.columns:
        if data_frame.dtypes[label] == object:
            data_frame[label] = data_frame[label].astype("str").astype("string")
    return data_frame

cat_images = cast_object_to_string(cat_images)

In [78]:
# cat_images = cat_images.reset_index()

In [33]:
cat_images.dtypes

id                 string
cws_id             string
updated_at         string
img_shape          string
img_vector         string
feature_vectors    string
dtype: object

In [34]:
cat_images.head()

,id,cws_id,updated_at,img_shape,img_vector,feature_vectors
0,4f5637ed-1bb9-4e51-979f-94213f0fbd63,1681491605-277,2023-05-15T02:51:17Z,"[128, 128, 3]","[-0.9944329104190696, -0.9947097270280661, -0....","[0.0, 2.044503927230835, 0.0, 0.0, 0.0, 0.0, 0..."
1,6b33d102-9497-43c7-a7a3-a28d6a9d1880,1681491000-6,2023-05-15T02:51:07Z,"[128, 128, 3]","[-0.99520184544406, -0.9951710880430603, -0.99...","[0.0, 2.0194919109344482, 0.0, 0.0, 0.0, 0.0, ..."
2,eec13c36-d5a2-4802-a508-517c2753663b,1681491462-213,2023-05-15T02:51:37Z,"[128, 128, 3]","[-0.9948327566320646, -0.9943713956170703, -0....","[0.0, 2.085791826248169, 0.0, 0.0, 0.0, 0.0, 0..."
3,8843e8e2-27c5-4bdf-966f-5e2cfc18a351,1681491248-117,2023-05-15T02:51:28Z,"[128, 128, 3]","[-0.9923106497500961, -0.9921876201460976, -0....","[0.0, 2.0698859691619873, 0.0, 0.0, 0.0, 0.0, ..."
4,d1f210b0-fd38-43fa-9f6b-880a8e2bf4de,1681491181-87,2023-05-15T02:51:11Z,"[128, 128, 3]","[-0.9969550173010381, -0.9965244136870435, -0....","[0.0, 2.1150710582733154, 0.0, 0.0, 0.0, 0.0, ..."


In [35]:
processed_folder = "../data/processed/auxiliary/"

if not os.path.exists(processed_folder):
    os.makedirs(processed_folder)

cat_images.to_pickle(f'{processed_folder}cat-images.pkl')